In [10]:
import import_hack
import steps
from data_store import sqlite_table_datastore as sqlds
from data_store import sqlite_table_one_to_many_datastore as sqlotmds
from data_store import stream_ndarray_adapter_datastore as snads
from quantization.pq_quantizer import PQQuantizer, restore_from_clusters
from transformer import transformers as tr
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
sqlite_db_path_clusters='ds_data\siftid_sift_samples0.1-clusters'
sqlite_table_name_clusters='clusterid_cluster'
ds_clusters = sqlds.SQLiteTableDataStore(sqlite_db_path_clusters, sqlite_table_name_clusters)
ds_clusters_ndarray_adapter = snads.StreamNdarrayAdapterDataStore(ds_clusters, detect_final_shape_by_first_elem=True)

In [22]:
with ds_clusters:
    print(ds_clusters.get_count())
    clusters=ds_clusters.get_items_sorted_by_ids()
    print(next(clusters).shape)
    print(next(clusters).shape)

2
(300, 64)
(300, 64)


In [23]:
clusters = ds_clusters_ndarray_adapter.get_items_sorted_by_ids()

In [32]:
print(clusters.shape)

(2, 300, 64)


In [37]:
pq_quantizer = restore_from_clusters(clusters)

(300, 64)
Initialization complete
Iteration  0, inertia 0.000
Converged at iteration 0
(300, 64)
Initialization complete
Iteration  0, inertia 0.000
Converged at iteration 0


In [38]:
print(pq_quantizer.get_cluster_centers().shape)

(2, 300, 64)


In [39]:
sqlite_db_path_in='ds_data\siftid_sift_imgid'
sqlite_table_name_in='siftid_sift_imgid'
sqlite_db_path_out='ds_data\imgid_productsiftbincount'
sqlite_table_name_out='imgid_productsiftbincount'

In [40]:
ds_in = sqlotmds.SQLiteTableOneToManyDataStore(sqlite_db_path_in, sqlite_table_name_in)
ds_out = sqlds.SQLiteTableDataStore(sqlite_db_path_out, sqlite_table_name_out)

In [42]:
with ds_in:
    items=ds_in.get_items_sorted_by_ids()
    print(next(items).shape)
    print(next(items).shape)

(379, 128)
(407, 128)


In [43]:
transformers=[tr.ArraysToProductBinCount(pq_quantizer)]

In [44]:
steps.transform_step(ds_in, transformers, ds_out)

In [45]:
with ds_out:
    print(ds_out.get_count())
    print(next(ds_out.get_items_sorted_by_ids()).shape)

999
(600,)
